# Twitter data collection


In [1]:
import pickle
#import os
#from collections import Counter
#import json
#import datetime as dt
import twitter
import sqlite3
import pandas as pd
import numpy as np

In [2]:
Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

In [3]:
#!pip install twitter

## Authorizing an application to access Twitter account data

In [4]:
auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth, retry=True)

## Collecting results in a sqlite database

In [5]:
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

In [6]:
# Create base
#------------------------------------------------------------
# Table: statuses
#------------------------------------------------------------
c.execute('''CREATE TABLE statuses(
        id                    Int NOT NULL ,
        coordinates           Varchar (250) ,
        created_at            Varchar (250) ,
        favorite_count        Int ,
        lang                  Varchar (25) ,
        place                 Varchar (2500) ,
        scopes                Varchar (2500) ,
        retweet_count         Int ,
        source                Varchar (2500) ,
        txt                   Varchar (250) ,
        withheld_in_countries Varchar (250) ,
        retweeted_status_id   Int ,
        in_reply_to_status_id Int ,
        in_reply_to_user_id   Int,
        quoted_status_id      Int ,
        user_id               Int ,
        PRIMARY KEY (id )
        );''')

#------------------------------------------------------------
# Table: users
#------------------------------------------------------------
c.execute('''CREATE TABLE users(
        id                    Int NOT NULL ,
        screen_name           Varchar (250) ,
        created_at            Varchar (100) ,
        description           Varchar (2500) ,
        favourites_count      Int ,
        followers_count       Int ,
        friends_count         Int ,
        lang                  Varchar (25) ,
        listed_count          Int ,
        location              Varchar (250) ,
        name                  Varchar (25) ,
        statuses_count        Int ,
        PRIMARY KEY (id )
        );''')

#------------------------------------------------------------
# Table: mentions
#------------------------------------------------------------

c.execute('''CREATE TABLE mentions(
        status_id Int NOT NULL ,
        user_id Int NOT NULL ,
        PRIMARY KEY (status_id ,user_id )
);''')

#------------------------------------------------------------
# Table: tags
#------------------------------------------------------------

c.execute('''CREATE TABLE tags(
        status_id Int NOT NULL ,
        htg    Varchar (140) NOT NULL ,
        PRIMARY KEY (status_id ,htg )
);''')

OperationalError: table statuses already exists

In [9]:
def add_user(u,c):
    #Create the user entity and main attributes
    c.execute('''INSERT INTO users(id,screen_name,created_at,description,favourites_count,
            followers_count,friends_count,lang,listed_count,
            location,name,statuses_count)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?) ''', 
          (str(u['id']),u['screen_name'],u['created_at'],u['description'],str(u['favourites_count']),
            str(u['followers_count']),str(u['friends_count']),u['lang'],str(u['listed_count']),
              u['location'],u['name'],str(u['statuses_count'])))

In [10]:
def add_status(s,c):
    #Create the status entity and main attributes
    c.execute('''
    INSERT INTO 
        statuses(id,created_at,favorite_count,
        lang,retweet_count,source,txt)
    VALUES (?, ?,? ,?  ,? ,? ,?)''', (str(s['id']),s['created_at'],str(s['favorite_count']),
        s['lang'],
        str(s['retweet_count']),
        s['source'], 
        s['text']))
    
    #Facultative attributes (None if absent)
    for k in ['in_reply_to_status_id','place','coordinates','in_reply_to_user_id']:
        if s[k] is not None:
            v=(k, str(s[k]), str(s['id']))
            c.execute('''
            UPDATE statuses
            SET %s = ?
            WHERE id = ?''' % k, (str(s[k]), str(s['id'])))
    
    #Facultative attributes (not in s if absent)
    
    for k in ['scopes','withheld_in_countries','quoted_status_id']:
        if k in s :
            c.execute('''
            UPDATE statuses
            SET %s=?
            WHERE id=?'''% k,(str(s[k]),str(s['id'])))

    #Other entities
    for k in ['retweeted_status','user']:
        if k in s :
            c.execute('''
                UPDATE statuses
                SET %s_id=?
                WHERE id=?''' % k, (str(s[k]['id']),str(s['id'])))

            
            
    ###########################################USERS##############################################
    if (c.execute("SELECT * FROM users WHERE id=?", (str(s['user']['id']),)).fetchone())==None:
        u=twitter_api.users.show(id=str(s['user']['id']))
        add_user(u,c)
    #else MAJ user

    if not s['in_reply_to_user_id']==None:
        if (c.execute("SELECT * FROM users WHERE id=?", (str(s['in_reply_to_user_id']),)).fetchone())==None:
            u=twitter_api.users.show(id=str(s['in_reply_to_user_id']))
            add_user(u,c)
        #else MAJ user
        
    for u in s['entities']['user_mentions']:
        if (c.execute('''SELECT * FROM mentions 
                    WHERE status_id = ? AND user_id = ?''', (str(s['id']),str(u['id']))).fetchone())==None:
            #We do not register the multiple mentions of a user in a tweet
            c.execute('''INSERT INTO mentions(status_id ,user_id)
                        VALUES (?, ?)''', (str(s['id']),str(u['id'])))

            if (c.execute('''SELECT * FROM users WHERE id=?''', (str(u['id']),)).fetchone())==None:
                u=twitter_api.users.show(id=str(u['id']))
                add_user(u,c)       

    ################################# HASHTAGS #################################################
    for h in s['entities']['hashtags']:
        if (c.execute('''SELECT * FROM tags 
                    WHERE status_id=? AND htg = ?''',(str(s['id']),h['text'].lower())).fetchone())==None:
            #Ajouter le htg 
            c.execute('''INSERT INTO tags(status_id, htg) VALUES (?,?)''',(str(s['id']),h['text'].lower()))

In [11]:
htgs = ['#egalim','#egalimentation','#EtatsGenerauxdelAlimentation','#EGA']
w = ["états généraux de l'alimentation"]
nb = 100
lg='fr'

In [12]:
###Collection of data
req=c.execute('''SELECT id FROM statuses ORDER by id DESC''').fetchone()

if req == None:
    sid=0
else : sid=req[0]
    
statuses=[]
for q in htgs :
    l_nw_st=100
    s=[]
    mid=10**20

    while l_nw_st==100:
        search_results = twitter_api.search.tweets(q=q, count=nb,  result_type='recent',  since_id=sid, max_id=mid)
        nw_st=search_results['statuses']
        l_nw_st=len(nw_st)
        s.extend(nw_st)
        if len(s)>0 : mid=s[-1]['id']-1   
    statuses.extend(s)


In [13]:
l_st=len(statuses)
print(l_st)

976


In [ ]:
[('{:-^100}'.format(s["user"]["screen_name"])+ '-'*5 + str(s['retweet_count'])+ '-'*2,s["text"],'') 
 for s in statuses][:5]

[('------------------------------------------GrainesInsoumis------------------------------------------------0--',
  '@Francois_Ruffin Dans le cadre des #EGAlim nous proposons une plus grande proximité entre producteurs et consommate… https://t.co/WqYALgJcMn',
  ''),
 ('------------------------------------------celineCuvillier------------------------------------------------23--',
  "RT @FNEasso: Cette décision est illogique et elle nuit au développement d'une agriculture durable. #EGAlim #SoutenonsLaBio https://t.co/QOk…",
  ''),
 ('--------------------------------------------guigsallain--------------------------------------------------23--',
  "RT @FNEasso: Cette décision est illogique et elle nuit au développement d'une agriculture durable. #EGAlim #SoutenonsLaBio https://t.co/QOk…",
  ''),
 ('--------------------------------------------famillemary--------------------------------------------------2--',
  'RT @UNAFapiculture: #EGAlim &gt;&gt; Nos propositions 👇 https://t.co/cW3dQLwtdz'

In [ ]:
for s in statuses:
    ls=len(c.execute('''SELECT * FROM statuses''').fetchall())
    lu=len(c.execute('''SELECT * FROM users''').fetchall())
    lh=len(c.execute('''SELECT * FROM tags''').fetchall())
    lm=len(c.execute('''SELECT * FROM mentions''').fetchall())
    print (ls,'  ',lu,'  ',lh,'  ',lm)
    #########################################STATUSES#############################################
    #main status
    if (c.execute("SELECT * FROM statuses WHERE id=?", (str(s['id']),)).fetchone())==None:
        add_status(s,c)
    
    #replied status
    if s['in_reply_to_status_id'] is not None:
        if (c.execute("SELECT * FROM statuses WHERE id=?", (str(s['in_reply_to_status_id']),)).fetchone())==None:
            r=twitter_api.statuses.show(id=s['in_reply_to_status_id'])
            add_status(r,c)        
                                           
    #quoted status
    if 'quoted_status_id' in s:
        if (c.execute("SELECT * FROM statuses WHERE id=?", (str(s['quoted_status_id']),)).fetchone())==None:
            q=twitter_api.statuses.show(id=s['quoted_status_id'])
            add_status(q,c)

     

3810    2150    6780    4915
3812    2150    6782    4916
3813    2151    6784    4917
3814    2152    6786    4918
3816    2154    6788    4919
3817    2154    6788    4919
3818    2155    6792    4921
3819    2156    6793    4922
3820    2157    6794    4923
3821    2158    6796    4924
3822    2159    6798    4925
3823    2160    6800    4926
3824    2161    6802    4927
3825    2162    6804    4928
3826    2163    6806    4929
3827    2164    6808    4930
3828    2165    6809    4931
3829    2166    6811    4932
3830    2167    6813    4933
3831    2168    6815    4934
3832    2169    6817    4935
3833    2170    6819    4936
3834    2171    6821    4937
3835    2172    6822    4938
3836    2173    6824    4939
3837    2174    6826    4940
3838    2175    6828    4941
3839    2175    6829    4942
3840    2175    6831    4943
3841    2175    6832    4944
3842    2176    6834    4945
3843    2177    6835    4946
3844    2178    6836    4947
3845    2178    6837    4947
3846    2179  

4106    2315    7238    5260
4107    2316    7240    5262
4108    2316    7241    5263
4109    2317    7244    5264
4110    2318    7247    5265
4111    2319    7249    5266
4112    2319    7252    5267
4113    2320    7255    5268
4114    2320    7257    5269
4115    2320    7259    5270
4117    2320    7261    5271
4118    2320    7262    5272
4119    2320    7264    5273
4120    2320    7266    5274
4121    2321    7268    5275
4122    2321    7269    5275
4123    2323    7271    5276
4124    2324    7273    5281
4125    2324    7276    5282
4126    2324    7277    5283
4127    2324    7279    5284
4128    2325    7280    5286
4129    2326    7281    5287
4130    2327    7282    5289
4131    2327    7284    5290
4132    2327    7286    5291
4133    2328    7287    5292
4134    2328    7288    5293
4135    2328    7289    5294
4136    2328    7290    5295
4137    2328    7292    5296
4138    2328    7295    5296
4139    2328    7296    5297
4140    2329    7298    5298
4141    2330  

4395    2396    7756    5567
4396    2396    7757    5568
4397    2396    7760    5569
4398    2396    7761    5570
4399    2396    7764    5571
4401    2396    7766    5572
4402    2397    7767    5573
4403    2397    7769    5577
4404    2397    7771    5578
4405    2398    7772    5581
4406    2398    7774    5582
4407    2398    7777    5583
4408    2398    7780    5584
4409    2398    7783    5585
4410    2399    7785    5586
4411    2399    7788    5587
4412    2399    7790    5588
4413    2399    7792    5589
4414    2399    7794    5591
4415    2399    7795    5593
4416    2400    7798    5594
4417    2400    7799    5595
4418    2400    7800    5596
4419    2400    7801    5598
4420    2401    7803    5599
4421    2401    7805    5599
4422    2401    7808    5600
4423    2401    7809    5601
4424    2401    7812    5602
4425    2402    7814    5604
4426    2402    7817    5605
4427    2402    7818    5607
4428    2402    7820    5609
4429    2402    7822    5611
4430    2402  

In [ ]:
ls=len(c.execute('''SELECT * FROM statuses''').fetchall())
lu=len(c.execute('''SELECT * FROM users''').fetchall())
lh=len(c.execute('''SELECT * FROM tags''').fetchall())
lm=len(c.execute('''SELECT * FROM mentions''').fetchall())
print (ls,'  ',lu,'  ',lh,'  ',lm)

In [ ]:
conn.commit()

In [ ]:
rows=c.execute('''SELECT users.screen_name, statuses.txt 
                    FROM users, statuses
                    WHERE 
                    statuses.user_id==users.id
                    AND users.name = ?''',('EGAlimentation',))

In [ ]:
print('{:-^120}'.format(rows.fetchone()[0]),rows.fetchone()[1])

## Data exploration

In [ ]:
rows=c.execute('''SELECT users.screen_name, statuses.txt 
                    FROM users, statuses
                    WHERE statuses.user_id==users.id''').fetchall()

In [ ]:
len(rows)

In [ ]:
list=[[name,status] 
     for (name,status) in rows]

In [ ]:
list[:5]

In [35]:
dict={'name':name,
     'status':status 
     for (name,status) in rows}

SyntaxError: invalid syntax (<ipython-input-35-6345032a7155>, line 3)